# Tarea: Hacer para N capas con un for

---
Imports

In [ ]:
import numpy as np
import pandas as pd

---
Código de la Capa

In [ ]:
class Capa:
    def __init__(self, num_neuronas, num_entradas, fun_act):
        self.peso_input = np.random.random((num_entradas+1, num_neuronas))
        self.fun_act = fun_act

    # Entra una matriz de 1 x n y sale una matriz de 1 x n
    def get_output(self, input):
        input = np.insert(input, 0, 1)
        if (input.shape[0] == self.peso_input.shape[0]):
            self.suma = np.matmul(input, self.peso_input)
            return self.step_function(self.suma)
        else: return "Ha habido un error: No coinciden el número de neuronas"

    # Decide qué hacer según la función de activación
    def step_function(self, input):
        if self.fun_act == 1:
            return self.sf_threshold(input)
        elif self.fun_act == 2:
            return self.sf_sigmoid(input)
        elif self.fun_act == 3:
            return self.sf_hyperbolic_tangent(input)
        elif self.fun_act == 4:
            return self.sf_relu(input)
        else:
            return "Ha habido un error: Función de activación incorrecta."

    def sf_threshold(self, input):
        output = input.copy()
        output[output<=0] = 0
        output[output>0] = 1
        return output

    def sf_sigmoid(self, input):
        return 1/(1+np.exp(-input))

    def sf_hyperbolic_tangent(self, input):
        return (np.exp(input) - np.exp(-input)) / (np.exp(input) + np.exp(-input))

    def sf_relu(self, input):
        return np.maximum([0], input)

    # Función para mostrar los datos de la capa
    def print(self):
        print("Inputs: \n", self.peso_input)
        print("Input shape: \n", self.peso_input.shape)
        print("F Act: \n", self.fun_act)

---

In [ ]:
class Red_Neuronal():
    def __init__(self,num_neuronas_capa, f_act_capa, num_inputs_1):
        self.lista_capas = []
        c = Capa(num_neuronas_capa[0], num_inputs_1, f_act_capa[0])
        self.lista_capas.append(c)
        for i in range(1, len(num_neuronas_capa)):
            c = Capa(num_neuronas_capa[i], num_neuronas_capa[i-1], f_act_capa[i])
            self.lista_capas.append(c)

    # Función para hacer los cálculos
    def test(self, inputs):
        for c in self.lista_capas:
            inputs = c.get_output(inputs)
        return inputs

    # Función para entrenar
    def train(self, input, out_esp , ratio):

        inputs, sumas = [], []
        for c in self.lista_capas:
            input = c.get_output(input)
            sumas.append(c.suma)
            inputs.append(input)

        # Derivadas de las funciones de activación
        df_act = self.f_act(sumas, 1)
        ecm = self.f_ecm(np.array(input), out_esp)
        decm = self.d_ecm(ecm)

        # Primera capa
        delta = decm * df_act # derivada f_coste * derivada f_activación

        pesos = self.lista_capas[-1].peso_input[1:]
        bias = self.lista_capas[-1].peso_input[0]

        self.lista_capas[-1].peso_input[1:] = pesos - delta * pesos * ratio
        self.lista_capas[-1].peso_input[0] = bias - delta * bias * ratio

        # Capas N
        for n in range(2, len(self.lista_capas)+1):
            pesos_tmp = pesos
            df_act = self.f_act(sumas, n)
            delta = delta @ pesos_tmp.transpose() * df_act # derivada f_coste * derivada f_activación * pesos anterior * derivada f_coste anterior

            pesos = self.lista_capas[-n].peso_input[1:]
            bias = self.lista_capas[-n].peso_input[0]

            # cambiar los self.pesos y self.bias
            self.lista_capas[-n].peso_input[1:] = pesos - np.array([inputs[-n] @ delta]).transpose() * ratio
            self.lista_capas[-n].peso_input[0] = bias - delta * bias * ratio

    # Función y derivada del ecm
    def f_ecm(self, r_esp, r_obt):
        ecm = ((r_esp - r_obt).sum()**2) / len(r_esp)
        return ecm

    def d_ecm(self, input):
        return np.sqrt(input)

    # Derivada según la función de activación
    def f_act(self, sumas, i):
        if (self.lista_capas[len(self.lista_capas)-i].fun_act) == 2:
            return self.d_sigmoid(sumas[len(sumas)-i])
        if (self.lista_capas[len(self.lista_capas)-i].fun_act) == 3:
            return self.d_hyperbolic_tangent(sumas[len(sumas)-i])
        if (self.lista_capas[len(self.lista_capas)-i].fun_act) == 4:
            return self.d_relu(sumas[len(sumas)-i])

    def d_sigmoid(self, input):
        return 1/(1+np.exp(-input)) * (1 - 1/(1+np.exp(-input)))

    def d_hyperbolic_tangent(self, input):
        return (2 / (np.exp(input) + np.exp(-input)))**2

    def d_relu(self, input):
        output = input.copy()
        output[output<=0] = 0
        output[output>0] = 1
        return output

    # Función para mostrar los datos de la red neuronal
    def print(self):
        c = 1
        for capa in self.lista_capas:
            print("Capa c: ")
            capa.print()
            print("---------")
            c += 1

In [ ]:
def recorrido(red, inputs, outputs, iteraciones, ra, num_tests):
    for i in range(1, iteraciones+1):
        for input, output in zip(inputs, outputs):
            red.train(input, output, ra)
        if i % num_tests == 0:
            resultados = []
            for input, output in zip(inputs, outputs):
                resultado = red.test(input)
                resultados.append(resultado)
            ecm = red.f_ecm(resultados, outputs)
            print(ecm)

---
Pruebas

In [ ]:
inputs = pd.read_csv("03_inputs.csv", header=None).values
outputs = pd.read_csv("03_outputs.csv", header=None).values
red = Red_Neuronal([8, 5, 7, 1], [3, 3, 3, 3], 2)
recorrido(red, inputs, outputs, 100, 0.001, 1)

In [ ]:
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output
# Display function
def show_training(nn, X, Y, costs, lr, iterations):
    loss = []

    for i in range(iterations):
        # Train neural network
        results = nn.train(X, Y, costs[1], lr)

        # Every 50 iterations, we check if network is learning showing the graphics
        if i % 50 == 0:
            # Save costs returns
            loss.append(costs[0](results, Y))

            # Show how our network is predicting
            res = 50
            _x0 = np.linspace(-1.5, 1.5, res)
            _x1 = np.linspace(-1.5, 1.5, res)
            _Y  = np.zeros((res, res))
            for i0, x0 in enumerate(_x0):
                for i1, x1 in enumerate(_x1):
                    _Y[i0, i1] = nn.test(np.array([[x0, x1]]))[0][0]
            
            plt.pcolormesh(_x0, _x1, _Y, cmap="coolwarm")
            plt.axis("equal")

            plt.scatter(X[Y[:,0] == 0, 0], X[Y[:,0] == 0, 1], c="skyblue")
            plt.scatter(X[Y[:,0] == 1, 0], X[Y[:,0] == 1, 1], c="salmon")

            clear_output(wait=True)
            plt.show()
            plt.plot(range(len(loss)), loss)
            plt.show()
            time.sleep(0.25)

In [ ]:
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output

def recorrido_grafico(red, inputs, outputs, iteraciones, ra, num_tests):
    for i in range(1, iteraciones+1):
        for input, output in zip(inputs, outputs):
            red.train(input, output, ra)
        if i % num_tests == 0:
            resultados = []
            for input, output in zip(inputs, outputs):
                resultado = red.test(input)
                resultados.append(resultado)
            ecm = red.f_ecm(resultados, outputs)
            print(ecm)

In [ ]:
inputs = pd.read_csv("03_inputs.csv", header=None).values
outputs = pd.read_csv("03_outputs.csv", header=None).values
red = Red_Neuronal([8, 5, 7, 1], [3, 3, 3, 3], 2)
recorrido_grafico(red, inputs, outputs, 0.001, 10)

In [ ]:
inputs = np.array([[0, 1, 1, 2, 1, 0]]).reshape(-1,1)
outputs = np.array([[0, 0, 1, 1, 0, 1]]).reshape(-1,1)
red = Red_Neuronal([1, 4, 3, 2, 1], [2, 3, 4, 3, 2], 1)
recorrido(inputs, outputs, 12000, 0.005, 500)